In [ ]:
import random
import weave
# Weave package now defaults to eager mode, but lazy mode required for this example notebook for now.
weave.use_lazy_execution()
from weave.legacy.weave.ecosystem import scenario

In [ ]:
# a = weave.save(5)
# b = a + 2
# weave.type_of(b)

In [ ]:
import math
def make_run(n_scenarios, n) -> list[scenario.ScenarioResult]:
    return [
        scenario.ScenarioResult(
            scenario_id=str(i),
            metric1=random.expovariate(n / 33),
            metric2=random.gauss(42, n / 100),
            metric3=random.triangular(n, 11),
            metric4=random.random() * random.random() * n,
            metric5=random.random(),
            metric6=random.betavariate(0.01, n / 2),
        )
        for i in range(n_scenarios)
    ]

In [ ]:
#panel_input = weave.save({'baseline': make_run(10), 'candidate': make_run(10)}, name='panel_input')

In [ ]:
#panel_input

In [ ]:
# make some runs

weave.save(make_run(100, 1), name='run1:latest')
weave.save(make_run(100, 2), name='run2:latest')
weave.save(make_run(100, 3), name='run3:latest')
weave.save(make_run(100, 4), name='run4:latest')
weave.save(make_run(100, 5), name='run5:latest')
pass

In [ ]:
# Ugly. We need to compute the Weave type right now.
# TODO: Fix
from weave.legacy.weave import infer_types
run_type = weave.types.List(infer_types.python_type_to_type(scenario.ScenarioResult))

import time
runs = weave.legacy.weave.ops.objects(run_type, 'latest', int(time.time()))

In [ ]:
panel = weave.legacy.weave.panels.Group(
    preferHorizontal=True,
    items={
        "sidebar": weave.legacy.weave.panels.Group(
            style="width: 200px; padding: 16px;",
            items={
                "baseline": weave.legacy.weave.panels.ObjectPicker(runs, label='baseline'),
                "candidate": weave.legacy.weave.panels.ObjectPicker(runs, label='candidate')
            }
        ),
        "main": lambda sidebar: weave.legacy.weave.panels.Group(
            items={
                "tables": weave.legacy.weave.panels.Group(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    items={
                        "baseline_table": weave.legacy.weave.panels.LabeledItem(
                            label="Baseline Table",
                            item=weave.legacy.weave.ops.execute(sidebar.config.items['baseline'].config.choice).get()
                        ),
                        "candidate_table": weave.legacy.weave.panels.LabeledItem(
                            label="Candidate Table",
                            item=weave.legacy.weave.ops.execute(sidebar.config.items['candidate'].config.choice).get()
                        )
                    }),
                "plots": 
                    weave.legacy.weave.panels.LabeledItem(
                        label='Scenario plots',
                        item=scenario.MetricsBankPanel(
                            weave.legacy.weave.ops.dict_(
                                baseline=weave.legacy.weave.ops.execute(sidebar.config.items['baseline'].config.choice).get(),
                                candidate=weave.legacy.weave.ops.execute(sidebar.config.items['candidate'].config.choice).get()
                            ))
                    )

            }
        )
    })
panel